In [3]:
import pandas as pd
import numpy as np
import random
import time
import datetime
import copy
import json
from tqdm import tqdm

In [63]:
#SETUP
dfUPB = pd.read_csv('dfUPB.csv')
dfPower = pd.read_csv('dfPower_List.csv')
dfTalent = pd.read_csv('dfTalent_List.csv')
dfContact = pd.read_csv('dfContact_List.csv')

FASERIP = {"F":"Ty","A":"Ty","S":"Ty","E":"Ty","R":"Ty","I":"Ty","P":"Ty"}

dict_faserip={"Sh0":0,"Fb":2,"Pr":4,"Ty":6,"Gd":10,"Ex":20,"Rm":30,"In":40,"Am":50,"Mn":75,"Un":100,"ShX":150,"ShY":200,"ShZ":500,"Cl1000":1000,"Cl3000":3000,"Cl5000":5000,"Beyond":1000000000}

faserip_index={"Sh0":0,"Fb":1,"Pr":2,"Ty":3,"Gd":4,"Ex":5,"Rm":6,"In":7,"Am":8,"Mn":9,"Un":10,"ShX":11,"ShY":12,"ShZ":13,"Cl1000":14,"Cl3000":15,"Cl5000":16,"Beyond":17}

newFASERIP = {'Name': '',
 'Identity': '',
 'Form': '',
 'Alt':'',
 'Alignment': '',
 'Type': '',
 'Size': '',
 'armour_name': '',
 'stated_ac': '',
 'body_armour': {"Physical":"Sh0","Energy":"Sh0"},           
 'resistances': {},                      
 'armor_bonus': '',
 'ac': '',
 'stated_hp': '',
 'hp': '',
 'expected_hp': '',
 'hp_fudge': '',
 'level': '',
 'hd': '',           
 #'F': 'Ty',
 #'A': 'Ty',
 #'S': 'Ty',
 #'E': 'Ty',
 #'R': 'Ty',
 #'I': 'Ty',
 #'P': 'Ty',
 'F': 0,
 'A': 0,
 'S': 0,
 'E': 0,
 'R': 0,
 'I': 0,
 'P': 0,
              
 'H': 1,
 'K': 1,
 'Res': 'Ty',
 'Pop': 'Ty',
 'Powers': {},
 'Powers_Rank': {},           
 'Powers_Adj': {},
 'Powers_Adj_Rank': {},           
 'Equipment': {},       
 'Equipment_Rank': {},           
 'Equipment_Adj': {},
 'Equipment_Adj_Rank': {},           
 'Vehicle': {},                      
 'Talents': {},
 'Talents_Adj': {},           
 'Contacts': {},
 'Contacts_Adj': {},
 'Weaknesses': {},
 'Weaknesses_Adj': {},           
 'Powers_Form': {},
 'Spd': 0,
 'Att': 0,
 'Attack':{'Blunt':{'S':'','R':'','A':'','AP':'','AB':''},'Edged':{'S':'','R':'','A':'','AP':''},\
           'Throwing Blunt':{'S':'','R':'','A':'','AP':''},'Throwing Edged':{'S':'','R':'','A':'','AP':''},\
           'Shooting':{'S':'','R':'','A':'','AP':''},\
           'Energy':{'S':'','R':'','A':'','AP':'','C':'','AB':''},'Force':{'S':'','R':'','A':'','AP':'','C':''},\
           'Grappling':{'S':'','R':'','A':'','AP':''},'Grabbing':{},'Charging':{},\
           'Mental':{'S':'','R':'','A':'','AP':'','C':''},'Magic':{'S':'','R':'','A':'','AP':'','C':''},\
           'Power':{'S':'','R':'','A':'','AP':'','C':'','AB':'','N':''}},
 'Defense':{'Escaping':{},'Dodging':{},'Evading':{},'Blocking':{},'Catching':{}, 'Ability':{},\
            'Blunt':{'AB':''},'Energy':{'AB':''},'Mental':{},'Magic':{}},
 'Dam': 0,
 'Climbing': 0,
 'Escaping': 0,
 'martial_arts':'',           
 'mook':0,           
 #bestiaryExtras here
 'attack_parameters': '',
 'CR': '',
 'xp': '',
 'regen': '',
 'healing_spells': '',
 'healing_dice': '',
 'healing_bonus': '',
 'sc_ability': '',
 'log': '',
 'proficiency': '',
 'initiative_bonus': '',
 'AB_F': '',
 'AB_A': '',
 'AB_S': '',
 'AB_E': '',
 'AB_R': '',
 'AB_I': '',
 'AB_P': ''
} 
for ability in FASERIP.keys():
    newFASERIP[ability] = 0
             


FASERIP_adjust = {}
FASERIP_adjust['Name'] = ""   
FASERIP_adjust['Identity'] = ""   
FASERIP_adjust['Form'] = "Normal Human"   
FASERIP_adjust['Alignment'] = ""   #for simulations some random string for a freeforall
FASERIP_adjust['Type'] = ""   
FASERIP_adjust['Size'] = ""   
for ability in FASERIP.keys():
    FASERIP_adjust[ability] = 0
FASERIP_adjust['H'] = 1   
FASERIP_adjust['K'] = 1   
FASERIP_adjust['Res'] = 0   
FASERIP_adjust['Pop'] = 0
FASERIP_adjust['Powers'] = 0  #becomes a list of stuff eventually - or dictionary?
FASERIP_adjust['Equipment'] = {}  #becomes a list of stuff eventually - or dictionary?
FASERIP_adjust['Talents'] = 0  #becomes a list of stuff eventually - or dictionary?
FASERIP_adjust['Contacts'] = 0  #becomes a list of stuff eventually - or dictionary?
FASERIP_adjust['Weaknesses'] = {}
FASERIP_adjust['Powers_Form'] = {}
FASERIP_adjust['Spd'] = 0
FASERIP_adjust['Att'] = 0
FASERIP_adjust['Dam'] = 0
FASERIP_adjust['Climbing'] = 0
FASERIP_adjust['Escaping'] = 0

all_forms = dfUPB['form'].unique()
all_forms_FASERIP = {}
for form in all_forms:
    #all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)

def column_shift(rank, shift):
    rank_list = []
    for key in faserip_index.keys():
        rank_list.append(key)
    rank_index = faserip_index[rank]
    new_index = rank_index + shift
    if new_index < 0: #lowest is zero
        new_index = 0
    if new_index > 13: #keep under cosmic
        new_index = 13

    return rank_list[new_index] #give back adjusted rank


def random_FASERIP():
    ability_roll = random.randint(0,6)
    ability_choice = list(FASERIP.keys())[ability_roll]
    return ability_choice


dfUPB

,roll,form,powers_min,power_category,talents_min,talents_max,talent_category,contacts_min,contacts_max,Roll,...,AMT,F,A,S,E,R,I,P,Res,Pop
0,1,Normal human,16,17000,6,8,18000,6,6,1,...,-1,0,0,0,0,0,0,0,1,0
1,2,Normal human,3,1000,0,3,1000,0,2,2,...,-1,0,0,0,0,0,0,0,1,0
2,3,Normal human,3,1000,0,3,1000,0,2,3,...,-1,0,0,0,0,0,0,0,1,0
3,4,Normal human,3,1000,0,3,1000,0,2,4,...,-1,0,0,0,0,0,0,0,1,0
4,5,Normal human,3,1000,0,3,1000,0,2,5,...,-1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,Energy,12,17000,4,8,18000,4,5,96,...,4,0,0,0,0,0,0,0,0,0
96,97,Ethereal,12,17000,4,8,18000,4,5,97,...,4,0,0,0,0,0,0,0,0,0
97,98,Undead,12,17000,4,8,18000,4,5,98,...,4,0,0,0,0,0,0,0,0,0
98,99,Mutant - Breed,14,17000,5,6,18000,5,5,99,...,4,0,0,0,0,0,0,0,0,0


# Random Generation

In [58]:
character_list = []

for make_faserip in tqdm(range(0,100), total=100):

    upb_form = dfUPB['form'].sample()
    upb_powers = dfUPB['powers_min'].sample()
    upb_talents = dfUPB['talents_min'].sample()
    upb_contacts = dfUPB['contacts_min'].sample()

    #get the list where >= the number and <= number + 1000
    power_categories = []
    powers_list = []

    for p in range(upb_powers.values[0]):
        c = dfUPB['power_category'].sample()
        power_categories.append(c.values[0])
        power_subset = dfPower.loc[(dfPower['roll'] >= c.values[0]) & (dfPower['roll'] < (c.values[0] + 1000))]    
        powers_list.append(power_subset.sample())

    talent_categories = []
    talent_list = []

    for p in range(upb_talents.values[0]):
        c = dfUPB['talent_category'].sample()
        talent_categories.append(c.values[0])
        talent_subset = dfTalent.loc[(dfTalent['roll'] >= c.values[0]) & (dfTalent['roll'] < (c.values[0] + 1000))]    
        talent_list.append(talent_subset.sample())   

    contact_list = []

    for p in range(upb_contacts.values[0]):
        c = dfContact.sample()
        contact_list.append(str(c.values[0][0]))   

    column_use = dfUPB.loc[dfUPB['Type'] == upb_form.values[0]]['Column'].values[0]

    ability_choice = str(random_FASERIP())  #this in each random character run

    ##adjustments
    form = 'Normal human'
    all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
    all_forms_FASERIP[form]['Res'] = 1

    form = 'Mutant - Induced'
    all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
    #all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form][ability_choice] = 1
    all_forms_FASERIP[form]
    form = 'Mutant - Random'
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['Res'] = -1
    all_forms_FASERIP[form]['Powers'] = 1
    all_forms_FASERIP[form]
    form = 'Mutant - Breed'
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['I'] = 1
    all_forms_FASERIP[form]['Contacts'] = 1

    form = 'Android'
    all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
    all_forms_FASERIP[form][ability_choice] = 1
    all_forms_FASERIP[form]['Pop'] = -1
    all_forms_FASERIP[form]['Powers'] = 1
    all_forms_FASERIP[form]['Contacts'] = 1

    form = 'Humanoid Race'
    all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
    all_forms_FASERIP[form][ability_choice] = 1
    all_forms_FASERIP[form]['Contacts'] = 1
    all_forms_FASERIP[form]['Powers'] = 1
    all_forms_FASERIP[form]['Res'] = {'Max':'Pr'}

    form = 'Surgical Composite'
    all_forms_FASERIP[form]['F'] = 1
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['Contacts'] = 1
    all_forms_FASERIP[form]['Res'] = {'Max':'Pr'}
    all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Powers_Form'] = {"Regeneration":"Fb"}
    all_forms_FASERIP[form]['Weaknesses'] = {"Resist Mental Domination":-1}
    all_forms_FASERIP[form]

    form = 'Modified Human - Organic'
    all_forms_FASERIP[form]['Powers_Form'] = {"Regeneration":"Fb"}

    form = 'Modified Human - Muscular'
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['Powers'] = -1

    form = 'Modified Human - Skeletal'
    all_forms_FASERIP[form]['Powers'] = -1
    all_forms_FASERIP[form]['Powers_Form'] = {"Resist: Physical":"Fb"}

    form = 'Modified Human - Extra Parts'
    #needs a list of Extra Parts to gain abilitiies from
    extra_parts = ['Arms','Tail','Wings','Fins','Antennae','Duplicate Organs']  #add powers from here
    random_index = random.randrange(len(extra_parts))
    extra_part = extra_parts[random_index]
    all_forms_FASERIP[form]['Powers'] = -1
    if extra_part == extra_parts[0]:
        all_forms_FASERIP[form]['F'] = 1
    elif extra_part == extra_parts[1]:
        all_forms_FASERIP[form]['Att'] = 1
    elif extra_part == extra_parts[2]:
        all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":0}
    elif extra_part == extra_parts[3]:
        all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Swimming":0}
    elif extra_part == extra_parts[3]:
        all_forms_FASERIP[form]['Powers_Form'] = {"True Sight":0}
    else:
        all_forms_FASERIP[form]['H'] = 2
    #print(all_forms_FASERIP[form])

    form = 'Demihuman - Centaur'
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['Climbing'] = -2

    form = 'Demihuman - Equiman'
    all_forms_FASERIP[form]['Dam'] = 1

    form = 'Demihuman - Faun'
    all_forms_FASERIP[form]['Dam'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"Domination":"Fb"}

    form = 'Demihuman - Felinoid'
    all_forms_FASERIP[form]['Powers_Form'] = {"Natural Weaponry":"S","Thermal Vision":"R"}

    form = 'Demihuman - Felinoid'
    all_forms_FASERIP[form]['Powers_Form'] = {"Natural Weaponry":"S","Thermal Vision":"Ex"}
    all_forms_FASERIP[form]['Climbing'] = 2

    form = 'Demihuman - Lupinoid'
    all_forms_FASERIP[form]['Pop'] = -1
    all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Olfactory":"Ex"}
    all_forms_FASERIP[form]['Climbing'] = 2

    form = 'Demihuman - Avian'
    all_forms_FASERIP[form]['Pop'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":"Ex"}

    form = 'Demihuman - Chiropteran'
    all_forms_FASERIP[form]['Pop'] = {'Max':'Fb'}
    all_forms_FASERIP[form]['Powers_Form'] = {"Sonar":"Gd","True Flight":"Ty"}

    form = 'Demihuman - Lamian'
    all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Escaping'] = 1
    all_forms_FASERIP[form]

    form = 'Demihuman - Merhuman'
    all_forms_FASERIP[form]['Pop'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"Water Freedom":"Un"}

    form = 'Demihuman - Harpy'
    all_forms_FASERIP[form]['F'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":"Ty"}

    form = 'Cyborg - Artificial limbs/organs'
    all_forms_FASERIP[form]['I'] = -1

    form = 'Cyborg - Exoskeletons'
    for ability in FASERIP.keys():
        #print(dfUPB['AMT'].sample().values[0])
        all_forms_FASERIP[form][ability] = int(dfUPB['AMT'].sample().values[0])

    form = 'Cyborg - Mechanical Body'
    all_forms_FASERIP[form]['I'] = -1
    all_forms_FASERIP[form]['P'] = -1
    all_forms_FASERIP[form]['Contacts'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"Resist: Poison":"Mn","Resist: Disease":"Mn"}

    form = 'Cyborg - Mechanically Augmented'
    all_forms_FASERIP[form]['Powers'] = -1
    all_forms_FASERIP[form]['Res'] = {'Min':'Gd'}

    form = 'Robot - Human Shape' 
    all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}

    form = 'Robot - Usuform'
    ##no changes

    form = 'Robot - Metamorphic'
    ##no changes #two forms, human and robot - for simulation -1CS - add this in the compund bucket

    form = 'Robot - Computer'
    all_forms_FASERIP[form]['F'] = -1
    all_forms_FASERIP[form]['R'] = 2
    all_forms_FASERIP[form]['Res'] = 1
    all_forms_FASERIP[form]['Weaknesses'] = {"Resist: Electricty":-1,"Resist: Magnetism":-1,"Resist: Phasing":-1}

    form = 'Angel/Demon' 
    side = random.randint(1,2)
    all_forms_FASERIP[form]['F'] = 1
    all_forms_FASERIP[form]['A'] = 1
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['R'] = 1
    all_forms_FASERIP[form]['I'] = 1
    all_forms_FASERIP[form]['P'] = 1
    if side == 1:
        all_forms_FASERIP[form]['Pop'] = 2
        all_forms_FASERIP[form]['Powers_Form'] = {"Artifact Creation-sword":"Ex"}
    else:
        all_forms_FASERIP[form]['Pop'] = -2
        all_forms_FASERIP[form]['Powers_Form'] = {"Fire Generation":"Gd", "True Invulnerability: Heat":"Cl1000", "True Invulnerability: Fire":"Cl1000"}

    form = 'Deity'
    all_forms_FASERIP[form]['F'] = 2
    all_forms_FASERIP[form]['A'] = 2
    all_forms_FASERIP[form]['S'] = 2
    all_forms_FASERIP[form]['E'] = 2
    all_forms_FASERIP[form]['R'] = 2
    all_forms_FASERIP[form]['I'] = 2
    all_forms_FASERIP[form]['P'] = 2
    all_forms_FASERIP[form]['Pop'] = 2

    form = 'Animal' 
    all_forms_FASERIP[form]['Res'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Powers'] = -1
    all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Hearing":"Gd", "Tracking":"Gd"}  #as examples, need a bunch of forms again

    form = 'Vegetable'
    all_forms_FASERIP[form]['F'] = -2
    all_forms_FASERIP[form]['E'] = 2
    all_forms_FASERIP[form]['Contacts'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"Absorption:Light":"Gd"}

    form = 'Abnormal Chemistry' 
    all_forms_FASERIP[form]['E'] = 1

    form = 'Mineral'
    all_forms_FASERIP[form]['H'] = 2
    all_forms_FASERIP[form]['Spd'] = -1

    form = 'Gaseous' 
    all_forms_FASERIP[form]['F'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Res'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Contacts'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Powers_Form'] = {"Resist:Physical":"Mn"}

    form = 'Liquid'
    all_forms_FASERIP[form]['Contacts'] = 1

    form = 'Energy'
    all_forms_FASERIP[form]['Powers_Form'] = {"Energy Field":"Fb", "Energy Emission":0, "Optional:Energy Control":0}
    all_forms_FASERIP[form]['Weakness'] = {"Plasma Control":-1}

    form = 'Ethereal'
    all_forms_FASERIP[form]['F'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Powers_Form'] = {"Resist:Physical":"Mn"}
    all_forms_FASERIP[form]['Weakness'] = {"Resist:Mental":-1,"Resist:Magical":-1}

    form = 'Undead'
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['Weakness'] = {"Holy:Symbol":"Ex"}

    adjustments = all_forms_FASERIP[upb_form.values[0]]

    #generate FASERIP
    character = copy.deepcopy(adjustments)
    character['Form'] = str(upb_form.values[0])

    print(character['Form'], str(column_use))
    for ability in FASERIP.keys():
        character[ability] = dfUPB[str(column_use)].sample().values[0]
        #print(character[ability])
        if type(adjustments[ability]) == int:
            character[ability] = column_shift(character[ability], adjustments[ability])
            #print(character[ability])
        else:
            #dict of max min eg Fighting of ghosts - only ability modified like that so 
            character[ability] = "Sh0"

    character['Res'] = "Ty"
    resource_adj = dfUPB['AMT'].sample().values[0]
    character['Pop'] = "Ty"

    character['H'] = dict_faserip[character['F']] + dict_faserip[character['A']] + dict_faserip[character['S']] + dict_faserip[character['E']]
    character['K'] = dict_faserip[character['R']] + dict_faserip[character['I']] + dict_faserip[character['P']]
    character['H'] = character['H'] * adjustments['H'] 

    if type(adjustments['Res']) == int:
        character['Res'] = column_shift(character['Res'], adjustments['Res'] + resource_adj)
    else:
        pass #dict with max and minimum
        faserip_index={"Sh0":0,"Fb":1,"Pr":2,"Ty":3,"Gd":4,"Ex":5,"Rm":6,"In":7,"Am":8,"Mn":9,"Un":10,"ShX":11,"ShY":12,"ShZ":13,"Cl1000":14,"Cl3000":15,"Cl5000":16,"Beyond":17}

        #check if dict -otherwise string from the above column shifting which is fine basic rank
        if not(type(character['Res'])) == dict:
            pass
        else:
            for key in character['Res'].keys():
                if key == "Max":
                    curindex = faserip_index[character['Res']]
                    checkindex = faserip_index[adjustments['Max']]
                    if checkindex > curindex:
                        character['Res'] = faserip_index[adjustments['Max']]
                else: #Min version
                    curindex = faserip_index[character['Res']]
                    checkindex = faserip_index[adjustments['Min']]
                    if checkindex < curindex:
                        character['Res'] = faserip_index[adjustments['Min']]

    if type(adjustments['Pop']) == int:
        character['Pop'] = column_shift(character['Pop'], adjustments['Pop'] + resource_adj)
    else:
        if not(type(character['Pop'])) == dict:
            pass
        else:
            for key in character['Pop'].keys():
                if key == "Max":
                    curindex = faserip_index[character['Pop']]
                    checkindex = faserip_index[adjustments['Max']]
                    if checkindex > curindex:
                        character['Pop'] = faserip_index[adjustments['Max']]
                else: #Min version
                    curindex = faserip_index[character['Pop']]
                    checkindex = faserip_index[adjustments['Min']]
                    if checkindex < curindex:
                        character['Pop'] = faserip_index[adjustments['Min']]

    talents = []
    for talent in talent_list:
        talents.append(str(talent['talent'].values[0]))
    character['Talents'] = talents

    if type(adjustments['Contacts']) == int:
        if adjustments['Contacts'] == 1:
            character['Contacts'] = ['Inital Form Contact']
        else:
            contacts = []
            for contact in contact_list:
                #print(contact)
                contacts.append(str(contact))
            character['Contacts'] = contacts

    else:
        character['Contacts'] = []

    ##got to put in power adjustments from forms - also two power powers    

    powers_adj = copy.deepcopy(powers_list)
    if type(adjustments['Powers']) == int:
        if adjustments['Powers'] < 0:
            #character['Contacts'] = ['Inital Form Contact']
            for i in range(abs(adjustments['Powers'])):
                del powers_adj[-1]
        elif adjustments['Powers'] > 0:
            #roll more powers
            pass

            extra_power_categories = []
            extra_powers_list = []

            for p in range(adjustments['Powers']):
                c = dfUPB['power_category'].sample()
                extra_power_categories.append(c.values[0])
                power_subset = dfPower.loc[(dfPower['roll'] >= c.values[0]) & (dfPower['roll'] < (c.values[0] + 1000))]    
                extra_powers_list.append(power_subset.sample())    

            powers_adj = powers_adj + extra_powers_list
        else:
            pass

    powers = []
    for power in powers_adj:
        powers.append(str(power['power'].values[0]))

    power_dict = {}
    for power in powers:    
        ##generate a rank
        power_rank = dfUPB[str(4)].sample().values[0]
        power_dict[power] = power_rank

    if not character['Powers_Form']:
        pass
    else:
        for pkey in character['Powers_Form']:
            power_dict[pkey] = character['Powers_Form'][pkey]

    character['Powers'] = power_dict

    ##need to add Powers_form into power_dict
    #character                               
    ## write out
    dateout = datetime.datetime.now()
    dateout = dateout.strftime("%Y_%d_%d_%H_%M_%S_%f")[:-3]
    char_dir = r'J:\CLONE\FASERIP' + "\\"
    with open(char_dir + dateout + '.json', 'w') as f:
        json.dump(character, f)

    df = pd.DataFrame([character])
    character_list.append(df)

dfCharacters = pd.concat(character_list)    
dfCharacters

  4%|███▎                                                                              | 4/100 [00:00<00:02, 39.97it/s]

Modified Human - Organic 1
Demihuman - Harpy 2
Normal human 2
Robot - Usuform 4
Normal human 2
Normal human 2
Normal human 2


 13%|██████████▌                                                                      | 13/100 [00:00<00:02, 37.25it/s]

Mutant - Breed 1
Modified Human - Extra Parts 1
Normal human 2
Cyborg - Artificial limbs/organs 4
Robot - Human Shape 4
Demihuman - Lamian 3
Animal 1
Modified Human - Extra Parts 1
Humanoid Race 5
Cyborg - Exoskeletons 4


 22%|█████████████████▊                                                               | 22/100 [00:00<00:01, 39.17it/s]

...

Normal human 2
Normal human 2
Mutant - Induced 1
Mutant - Random 1
Normal human 2


 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [00:02<00:00, 35.96it/s]

Humanoid Race 5
Mutant - Random 1
Normal human 2
Robot - Human Shape 4
Modified Human - Organic 1
Vegetable 1
Normal human 2
Cyborg - Exoskeletons 4
Normal human 2


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 33.77it/s]


,Name,Identity,Form,Alt,Alignment,Type,Size,armour_name,stated_ac,body_armour,...,proficiency,initiative_bonus,AB_F,AB_A,AB_S,AB_E,AB_R,AB_I,AB_P,Weakness
0,,,Modified Human - Organic,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Demihuman - Harpy,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Normal human,NaN,,,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,,,Robot - Usuform,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Normal human,NaN,,,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,,,Modified Human - Organic,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Vegetable,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Normal human,NaN,,,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,,,Cyborg - Exoskeletons,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN


In [71]:
####updated version 20220114

character_list = []

for make_faserip in tqdm(range(0,100), total=100):

    upb_form = dfUPB['form'].sample()
    upb_powers = dfUPB['powers_min'].sample()
    upb_talents = dfUPB['talents_min'].sample()
    upb_contacts = dfUPB['contacts_min'].sample()

    #get the list where >= the number and <= number + 1000
    power_categories = []
    powers_list = []

    for p in range(upb_powers.values[0]):
        c = dfUPB['power_category'].sample()
        power_categories.append(c.values[0])
        power_subset = dfPower.loc[(dfPower['roll'] >= c.values[0]) & (dfPower['roll'] < (c.values[0] + 1000))]    
        powers_list.append(power_subset.sample())

    talent_categories = []
    talent_list = []

    for p in range(upb_talents.values[0]):
        c = dfUPB['talent_category'].sample()
        talent_categories.append(c.values[0])
        talent_subset = dfTalent.loc[(dfTalent['roll'] >= c.values[0]) & (dfTalent['roll'] < (c.values[0] + 1000))]    
        talent_list.append(talent_subset.sample())   

    contact_list = []

    for p in range(upb_contacts.values[0]):
        c = dfContact.sample()
        contact_list.append(str(c.values[0][0]))   

    column_use = dfUPB.loc[dfUPB['Type'] == upb_form.values[0]]['Column'].values[0]

    ability_choice = str(random_FASERIP())  #this in each random character run

    ##adjustments
    form = 'Normal human'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Res'] = 1

    form = 'Mutant - Induced'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    #all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form][ability_choice] = 1
    all_forms_FASERIP[form]
    form = 'Mutant - Random'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['Res'] = -1
    all_forms_FASERIP[form]['Powers'] = 1
    all_forms_FASERIP[form]
    
    form = 'Mutant - Breed'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['I'] = 1
    all_forms_FASERIP[form]['Contacts'] = 1

    form = 'Android'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form][ability_choice] = 1
    all_forms_FASERIP[form]['Pop'] = -1
    all_forms_FASERIP[form]['Powers'] = 1
    all_forms_FASERIP[form]['Contacts'] = 1

    form = 'Humanoid Race'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form][ability_choice] = 1
    all_forms_FASERIP[form]['Contacts'] = 1
    all_forms_FASERIP[form]['Powers'] = 1
    all_forms_FASERIP[form]['Res'] = {'Max':'Pr'}

    form = 'Surgical Composite'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['F'] = 1
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['Contacts'] = 1
    all_forms_FASERIP[form]['Res'] = {'Max':'Pr'}
    all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Powers_Form'] = {"Regeneration":"Fb"}
    all_forms_FASERIP[form]['Weaknesses'] = {"Resist Mental Domination":-1}
    all_forms_FASERIP[form]

    form = 'Modified Human - Organic'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Powers_Form'] = {"Regeneration":"Fb"}

    form = 'Modified Human - Muscular'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['Powers'] = -1

    form = 'Modified Human - Skeletal'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Powers'] = -1
    all_forms_FASERIP[form]['Powers_Form'] = {"Resist: Physical":"Fb"}

    form = 'Modified Human - Extra Parts'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    #needs a list of Extra Parts to gain abilitiies from
    extra_parts = ['Arms','Tail','Wings','Fins','Antennae','Duplicate Organs']  #add powers from here
    random_index = random.randrange(len(extra_parts))
    extra_part = extra_parts[random_index]
    all_forms_FASERIP[form]['Powers'] = -1
    if extra_part == extra_parts[0]:
        all_forms_FASERIP[form]['F'] = 1
    elif extra_part == extra_parts[1]:
        all_forms_FASERIP[form]['Att'] = 1
    elif extra_part == extra_parts[2]:
        all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":0}
    elif extra_part == extra_parts[3]:
        all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Swimming":0}
    elif extra_part == extra_parts[3]:
        all_forms_FASERIP[form]['Powers_Form'] = {"True Sight":0}
    else:
        all_forms_FASERIP[form]['H'] = 2
    #print(all_forms_FASERIP[form])

    form = 'Demihuman - Centaur'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['Climbing'] = -2

    form = 'Demihuman - Equiman'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Dam'] = 1

    form = 'Demihuman - Faun'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Dam'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"Domination":"Fb"}

    form = 'Demihuman - Felinoid'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Powers_Form'] = {"Natural Weaponry":"S","Thermal Vision":"R"}

    form = 'Demihuman - Felinoid'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Powers_Form'] = {"Natural Weaponry":"S","Thermal Vision":"Ex"}
    all_forms_FASERIP[form]['Climbing'] = 2

    form = 'Demihuman - Lupinoid'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Pop'] = -1
    all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Olfactory":"Ex"}
    all_forms_FASERIP[form]['Climbing'] = 2

    form = 'Demihuman - Avian'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Pop'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":"Ex"}

    form = 'Demihuman - Chiropteran'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Pop'] = {'Max':'Fb'}
    all_forms_FASERIP[form]['Powers_Form'] = {"Sonar":"Gd","True Flight":"Ty"}

    form = 'Demihuman - Lamian'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Escaping'] = 1
    all_forms_FASERIP[form]

    form = 'Demihuman - Merhuman'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Pop'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"Water Freedom":"Un"}

    form = 'Demihuman - Harpy'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['F'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":"Ty"}

    form = 'Cyborg - Artificial limbs/organs'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['I'] = -1

    form = 'Cyborg - Exoskeletons'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    for ability in FASERIP.keys():
        #print(dfUPB['AMT'].sample().values[0])
        all_forms_FASERIP[form][ability] = int(dfUPB['AMT'].sample().values[0])

    form = 'Cyborg - Mechanical Body'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['I'] = -1
    all_forms_FASERIP[form]['P'] = -1
    all_forms_FASERIP[form]['Contacts'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"Resist: Poison":"Mn","Resist: Disease":"Mn"}

    form = 'Cyborg - Mechanically Augmented'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Powers'] = -1
    all_forms_FASERIP[form]['Res'] = {'Min':'Gd'}

    form = 'Robot - Human Shape' 
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}

    form = 'Robot - Usuform'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    ##no changes

    form = 'Robot - Metamorphic'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    ##no changes #two forms, human and robot - for simulation -1CS - add this in the compund bucket

    form = 'Robot - Computer'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['F'] = -1
    all_forms_FASERIP[form]['R'] = 2
    all_forms_FASERIP[form]['Res'] = 1
    all_forms_FASERIP[form]['Weaknesses'] = {"Resist: Electricty":-1,"Resist: Magnetism":-1,"Resist: Phasing":-1}

    form = 'Angel/Demon' 
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    side = random.randint(1,2)
    all_forms_FASERIP[form]['F'] = 1
    all_forms_FASERIP[form]['A'] = 1
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['R'] = 1
    all_forms_FASERIP[form]['I'] = 1
    all_forms_FASERIP[form]['P'] = 1
    if side == 1:
        all_forms_FASERIP[form]['Pop'] = 2
        all_forms_FASERIP[form]['Powers_Form'] = {"Artifact Creation-sword":"Ex"}
    else:
        all_forms_FASERIP[form]['Pop'] = -2
        all_forms_FASERIP[form]['Powers_Form'] = {"Fire Generation":"Gd", "True Invulnerability: Heat":"Cl1000", "True Invulnerability: Fire":"Cl1000"}

    form = 'Deity'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['F'] = 2
    all_forms_FASERIP[form]['A'] = 2
    all_forms_FASERIP[form]['S'] = 2
    all_forms_FASERIP[form]['E'] = 2
    all_forms_FASERIP[form]['R'] = 2
    all_forms_FASERIP[form]['I'] = 2
    all_forms_FASERIP[form]['P'] = 2
    all_forms_FASERIP[form]['Pop'] = 2

    form = 'Animal' 
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Res'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Powers'] = -1
    all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Hearing":"Gd", "Tracking":"Gd"}  #as examples, need a bunch of forms again

    form = 'Vegetable'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['F'] = -2
    all_forms_FASERIP[form]['E'] = 2
    all_forms_FASERIP[form]['Contacts'] = 1
    all_forms_FASERIP[form]['Powers_Form'] = {"Absorption:Light":"Gd"}

    form = 'Abnormal Chemistry' 
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['E'] = 1

    form = 'Mineral'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['H'] = 2
    all_forms_FASERIP[form]['Spd'] = -1

    form = 'Gaseous' 
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['F'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Res'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Contacts'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Powers_Form'] = {"Resist:Physical":"Mn"}

    form = 'Liquid'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Contacts'] = 1

    form = 'Energy'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['Powers_Form'] = {"Energy Field":"Fb", "Energy Emission":0, "Optional:Energy Control":0}
    all_forms_FASERIP[form]['Weakness'] = {"Plasma Control":-1}

    form = 'Ethereal'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['F'] = {'Max':'Sh0'}
    all_forms_FASERIP[form]['Powers_Form'] = {"Resist:Physical":"Mn"}
    all_forms_FASERIP[form]['Weakness'] = {"Resist:Mental":-1,"Resist:Magical":-1}

    form = 'Undead'
    all_forms_FASERIP[form] = copy.deepcopy(newFASERIP)
    all_forms_FASERIP[form]['S'] = 1
    all_forms_FASERIP[form]['E'] = 1
    all_forms_FASERIP[form]['Weakness'] = {"Holy:Symbol":"Ex"}

    adjustments = all_forms_FASERIP[upb_form.values[0]]

    #generate FASERIP
    character = copy.deepcopy(adjustments)
    character['Form'] = str(upb_form.values[0])

    #print(character['Form'], str(column_use))
    for ability in FASERIP.keys():
        character[ability] = dfUPB[str(column_use)].sample().values[0]
        #print(character[ability])
        if type(adjustments[ability]) == int:
            character[ability] = column_shift(character[ability], adjustments[ability])
            #print(character[ability])
        else:
            #dict of max min eg Fighting of ghosts - only ability modified like that so 
            character[ability] = "Sh0"

    character['Res'] = "Ty"
    resource_adj = dfUPB['AMT'].sample().values[0]
    character['Pop'] = "Ty"

    character['H'] = dict_faserip[character['F']] + dict_faserip[character['A']] + dict_faserip[character['S']] + dict_faserip[character['E']]
    character['K'] = dict_faserip[character['R']] + dict_faserip[character['I']] + dict_faserip[character['P']]
    character['H'] = character['H'] * adjustments['H'] 

    if type(adjustments['Res']) == int:
        character['Res'] = column_shift(character['Res'], adjustments['Res'] + resource_adj)
    else:
        pass #dict with max and minimum
        faserip_index={"Sh0":0,"Fb":1,"Pr":2,"Ty":3,"Gd":4,"Ex":5,"Rm":6,"In":7,"Am":8,"Mn":9,"Un":10,"ShX":11,"ShY":12,"ShZ":13,"Cl1000":14,"Cl3000":15,"Cl5000":16,"Beyond":17}

        #check if dict -otherwise string from the above column shifting which is fine basic rank
        if not(type(character['Res'])) == dict:
            pass
        else:
            for key in character['Res'].keys():
                if key == "Max":
                    curindex = faserip_index[character['Res']]
                    checkindex = faserip_index[adjustments['Max']]
                    if checkindex > curindex:
                        character['Res'] = faserip_index[adjustments['Max']]
                else: #Min version
                    curindex = faserip_index[character['Res']]
                    checkindex = faserip_index[adjustments['Min']]
                    if checkindex < curindex:
                        character['Res'] = faserip_index[adjustments['Min']]

    if type(adjustments['Pop']) == int:
        character['Pop'] = column_shift(character['Pop'], adjustments['Pop'] + resource_adj)
    else:
        if not(type(character['Pop'])) == dict:
            pass
        else:
            for key in character['Pop'].keys():
                if key == "Max":
                    curindex = faserip_index[character['Pop']]
                    checkindex = faserip_index[adjustments['Max']]
                    if checkindex > curindex:
                        character['Pop'] = faserip_index[adjustments['Max']]
                else: #Min version
                    curindex = faserip_index[character['Pop']]
                    checkindex = faserip_index[adjustments['Min']]
                    if checkindex < curindex:
                        character['Pop'] = faserip_index[adjustments['Min']]

    talents = []
    for talent in talent_list:
        talents.append(str(talent['talent'].values[0]))
    character['Talents'] = talents

    if type(adjustments['Contacts']) == int:
        if adjustments['Contacts'] == 1:
            character['Contacts'] = ['Inital Form Contact']
        else:
            contacts = []
            for contact in contact_list:
                print(contact)
                character['Contacts'][contact] = 1
                #contacts.append(str(contact))
            #character['Contacts'] = contacts

    else:
        #character['Contacts'] = []
        for contact in contact_list:
            print(contact)
            character['Contacts'][contact] = 1
        

    ##got to put in power adjustments from forms - also two power powers    

    powers_adj = copy.deepcopy(powers_list)
    if type(adjustments['Powers']) == int:
        if adjustments['Powers'] < 0:
            #character['Contacts'] = ['Inital Form Contact']
            for i in range(abs(adjustments['Powers'])):
                del powers_adj[-1]
        elif adjustments['Powers'] > 0:
            #roll more powers
            pass

            extra_power_categories = []
            extra_powers_list = []

            for p in range(adjustments['Powers']):
                c = dfUPB['power_category'].sample()
                extra_power_categories.append(c.values[0])
                power_subset = dfPower.loc[(dfPower['roll'] >= c.values[0]) & (dfPower['roll'] < (c.values[0] + 1000))]    
                extra_powers_list.append(power_subset.sample())    

            powers_adj = powers_adj + extra_powers_list
        else:
            pass

    powers = []
    for power in powers_adj:
        powers.append(str(power['power'].values[0]))

    power_dict = {}
    for power in powers:    
        ##generate a rank
        power_rank = dfUPB[str(4)].sample().values[0]
        power_dict[power] = power_rank

    if not character['Powers_Form']:
        pass
    else:
        for pkey in character['Powers_Form']:
            power_dict[pkey] = character['Powers_Form'][pkey]

    character['Powers'] = power_dict

    ##need to add Powers_form into power_dict
    #character                               
    ## write out
    dateout = datetime.datetime.now()
    dateout = dateout.strftime("%Y_%d_%d_%H_%M_%S_%f")[:-3]
    char_dir = r'J:\CLONE\FASERIP' + "\\"
    with open(char_dir + dateout + '.json', 'w') as f:
        json.dump(character, f)

    df = pd.DataFrame([character])
    character_list.append(df)

dfCharacters = pd.concat(character_list)    
dfCharacters

  8%|██████▌                                                                           | 8/100 [00:00<00:02, 36.24it/s]

Mythology
Medicine
Chemistry
Electronics
Physics
Medicine
Chemistry


 16%|████████████▉                                                                    | 16/100 [00:00<00:02, 33.16it/s]

Artist/Performer
Journalism
Engineering
Law


 24%|███████████████████▍                                                             | 24/100 [00:00<00:02, 35.28it/s]

...

Political-Other
Psychiatry
Religion
Political-National
Electronics
Military
Journalism
Engineering
Medicine
Military
Computers
Political-Planetary
Military


 95%|████████████████████████████████████████████████████████████████████████████▉    | 95/100 [00:03<00:00, 30.03it/s]

Hero
Political-State
Electronics
Journalism
Military
Chemistry
Occult Lore
Detective/Espionage
Law-Enforcement
Chemistry
Computers
Law
Political-International


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 28.92it/s]


,Name,Identity,Form,Alt,Alignment,Type,Size,armour_name,stated_ac,body_armour,...,proficiency,initiative_bonus,AB_F,AB_A,AB_S,AB_E,AB_R,AB_I,AB_P,Weakness
0,,,Demihuman - Chiropteran,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Surgical Composite,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Normal human,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Modified Human - Extra Parts,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Robot - Human Shape,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,,,Normal human,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Mutant - Induced,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Robot - Metamorphic,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
0,,,Mineral,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN


In [72]:
character
dfCharacters.reset_index(inplace=True, drop=True)
dfCharacters

,Name,Identity,Form,Alt,Alignment,Type,Size,armour_name,stated_ac,body_armour,...,proficiency,initiative_bonus,AB_F,AB_A,AB_S,AB_E,AB_R,AB_I,AB_P,Weakness
0,,,Demihuman - Chiropteran,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
1,,,Surgical Composite,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
2,,,Normal human,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
3,,,Modified Human - Extra Parts,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
4,,,Robot - Human Shape,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,,,Normal human,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
96,,,Mutant - Induced,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
97,,,Robot - Metamorphic,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN
98,,,Mineral,,,,,,,"{'Physical': 'Sh0', 'Energy': 'Sh0'}",...,,,,,,,,,,NaN


In [74]:
dfCharacters.to_csv('Random_Characters2.csv', index=False)

In [51]:
character

{'Name': '',
 'Identity': '',
 'Form': 'Humanoid Race',
 'Alignment': '',
 'Type': '',
 'Size': '',
 'F': 'Ex',
 'A': 'Ex',
 'S': 'Mn',
 'E': 'Ex',
 'R': 'Gd',
 'I': 'Am',
 'P': 'In',
 'H': 135,
 'K': 100,
 'Res': 'Ty',
 'Pop': 'Ex',
 'Powers': {'Penetration Vision': 'Ex',
  'Armor Skin': 'Gd',
  'Magnetism': 'In',
  'Disruption': 'Gd',
  'Hyper-Endurance': 'Rm'},
 'Equipment': {},
 'Talents': ['Radio Astronomy'],
 'Contacts': ['Inital Form Contact'],
 'Weaknesses': {},
 'Powers_Form': {},
 'Spd': 0,
 'Att': 0,
 'Dam': 0,
 'Climbing': 0,
 'Escaping': 0}

In [21]:
all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
for ability in FASERIP.keys():
    #print(dfUPB['AMT'].sample().values[0])
    all_forms_FASERIP[form][ability] = column_shift(all_forms_FASERIP[form][ability], dfUPB['AMT'].sample().values[0])
    print(all_forms_FASERIP[form][ability])
    

KeyError: 0